# QuackOSM Basic Usage

**QuackOSM** exposes some basic functions in the main Python module. Full documentation for them is available [here](../../api/QuackOSM/).

This notebook will show how to use the library in a few simple scenarios.

To learn more about `CLI` usage, see [this page](../command_line_interface/). The `help` page for the CLI is available [here](../../api/CLI/).

To learn more details about `PbfFileReader` class, see [this page](../pbf_file_reader/), or [documentation page](../../api/PbfFileReader/).

In [ ]:
import quackosm as qosm

## Reading existing PBF file to GeoDataFrame

In [ ]:
qosm.convert_pbf_to_geodataframe("https://download.geofabrik.de/europe/monaco-latest.osm.pbf")

## Transforming existing PBF file to GeoParquet

In [ ]:
qosm.convert_pbf_to_parquet("https://download.geofabrik.de/europe/monaco-latest.osm.pbf")

## Find an OSM PBF extract file using text and read it as GeoDataFrame

Read more about OSM extracts [here](../advanced_examples/osm_extracts/).

In [ ]:
qosm.convert_osm_extract_to_geodataframe("Vatican City")

## Find an OSM PBF extract file using text and transform it to a GeoParquet

In [ ]:
qosm.convert_osm_extract_to_parquet("Vatican City")

## Get OSM data for a given geometry as a GeoDataFrame

In [ ]:
area = qosm.geocode_to_geometry("Songpa-gu, Seoul")
qosm.convert_geometry_to_geodataframe(area)

## Save OSM data for a given geometry as a GeoParquet

In [ ]:
qosm.convert_geometry_to_parquet(area)

## More advanced examples

### Filter out data based on geometry from existing PBF file

In [ ]:
area = qosm.geocode_to_geometry("Monaco-Ville, Monaco")
gdf = qosm.convert_pbf_to_geodataframe(
    "https://download.geofabrik.de/europe/monaco-latest.osm.pbf", geometry_filter=area
)
gdf

Plot downloaded data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd

fig = plt.figure(figsize=(10, 10))
ax = fig.subplots()

gdf.plot(ax=ax, markersize=1, zorder=1, alpha=0.4)
gdf.boundary.plot(ax=ax, markersize=0, zorder=1, alpha=0.8)
gpd.GeoSeries([area], crs=4326).plot(
    ax=ax,
    color=(0, 0, 0, 0),
    zorder=2,
    hatch="///",
    edgecolor="orange",
    linewidth=1.5,
)

blue_patch = mpatches.Patch(color="C0", alpha=0.8, label="OSM features")
orange_patch = mpatches.Patch(
    facecolor=(0, 0, 0, 0), edgecolor="orange", hatch="///", linewidth=1.5, label="Geometry filter"
)
ax.legend(handles=[blue_patch, orange_patch], loc="lower right")
plt.show()

### Get all bicycle sharing stations in Barcelona

You can read more about OSM tags filtering [here](../advanced_examples/osm_tags_filter/).

In [ ]:
area = qosm.geocode_to_geometry("Barcelona")
gdf = qosm.convert_geometry_to_geodataframe(
    area, tags_filter={"amenity": "bicycle_rental"}
)
gdf

Show downloaded data on a map

In [ ]:
m = gdf.explore(color="orangered", tiles="CartoDB positron")
gpd.GeoSeries([area], crs=4326).boundary.explore(m=m)

### Save the result GeoParquet with WKT geometry

In [ ]:
qosm.convert_pbf_to_parquet(
    "https://download.geofabrik.de/europe/monaco-latest.osm.pbf",
    save_as_wkt=True,
    sort_result=False, # sorting is disabled for wkt output
)

### Specify result file path

In [ ]:
qosm.convert_geometry_to_parquet(
    area, result_file_path='barcelona_osm_output.parquet'
)

### Force recalculation of the result

By default, running the same command twice will result in reusing the saved GeoParquet file. You can force QuackOSM to recalculate the data.

In [ ]:
qosm.convert_pbf_to_parquet(
    "https://download.geofabrik.de/europe/monaco-latest.osm.pbf", ignore_cache=True
)

### Result file sorting
By default, **QuackOSM** sorts the result file by geometry using Hilbert curve to make it smaller. It adds some time to the overall execution, but can significantly reduce the file size.

Sorting can be disabled by the user.

In [ ]:
unsorted_pq = qosm.convert_geometry_to_parquet(
    area, tags_filter={"building": True}, sort_result=False
)

In [ ]:
sorted_pq = qosm.convert_geometry_to_parquet(
    area, tags_filter={"building": True}, sort_result=True
)

In [ ]:
unsorted_pq, sorted_pq

In [ ]:
import geopandas as gpd
from matplotlib import pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

gpd.read_parquet(unsorted_pq).reset_index().reset_index().plot(
    column="index", ax=ax1, cmap="jet", markersize=1
)
gpd.read_parquet(sorted_pq).reset_index().reset_index().plot(
    column="index", ax=ax2, cmap="jet", markersize=1
)

unsorted_size = unsorted_pq.stat().st_size
sorted_size = sorted_pq.stat().st_size

ax1.set_title(f"Unsorted: {unsorted_size} bytes")
ax2.set_title(
    f"Sorted: {sorted_size} bytes ({100 - (100 * sorted_size) / unsorted_size:.2f}% reduction)"
)

plt.show()